## this is to run the model with the best regression and RMSE score for the model determined before 

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter


In [2]:
# Load the data
file_path = Path('happiness_joined.csv')
happiness_joined_df = pd.read_csv(file_path,encoding="ISO-8859-1")#,index_col=0)
# happiness_joined_df = df.loc[:, columns].copy()

# df.reset_index(inplace=True, drop=True)

happiness_joined_df.head()


,country_code,country_name,life_ladder,democracy_index,consumer_price_index,gender_ratio_males_per100_female,infant_mortality_per1000_births,life_expectancy,per_capita_gdp_dollars,population_density,safe_drinking_water_access_pct,seats_held_by_women_pct,unemployment_rate
0,8,Albania,4.995,58.9,119.1,103.7,8.0,78.4,5303,105.1,70.7,29.3,12.8
1,12,Algeria,4.745,40.1,151.4,102.1,21.2,76.6,3976,18.1,72.4,25.8,11.5
2,51,Armenia,5.488,55.4,129.2,88.8,10.8,74.9,4623,103.9,86.9,24.2,16.6
3,40,Austria,7.195,82.9,118.1,97.0,3.2,81.4,49701,108.7,98.9,37.2,4.8
4,31,Azerbaijan,5.173,27.5,156.9,99.7,20.8,72.8,4782,121.6,88.3,16.8,6.0


In [3]:
happiness_joined_df[['country_code','democracy_index','consumer_price_index','gender_ratio_males_per100_female','infant_mortality_per1000_births','life_expectancy','per_capita_gdp_dollars','population_density','safe_drinking_water_access_pct','seats_held_by_women_pct','unemployment_rate']] = happiness_joined_df[['country_code','democracy_index','consumer_price_index','gender_ratio_males_per100_female','infant_mortality_per1000_births','life_expectancy','per_capita_gdp_dollars','population_density','safe_drinking_water_access_pct','seats_held_by_women_pct','unemployment_rate']].replace({'\$': '', ',': ''}, regex=True).astype(float)
#
happiness_joined_df.head()
# # REF: https://pbpython.com/currency-cleanup.html

,country_code,country_name,life_ladder,democracy_index,consumer_price_index,gender_ratio_males_per100_female,infant_mortality_per1000_births,life_expectancy,per_capita_gdp_dollars,population_density,safe_drinking_water_access_pct,seats_held_by_women_pct,unemployment_rate
0,8.0,Albania,4.995,58.9,119.1,103.7,8.0,78.4,5303.0,105.1,70.7,29.3,12.8
1,12.0,Algeria,4.745,40.1,151.4,102.1,21.2,76.6,3976.0,18.1,72.4,25.8,11.5
2,51.0,Armenia,5.488,55.4,129.2,88.8,10.8,74.9,4623.0,103.9,86.9,24.2,16.6
3,40.0,Austria,7.195,82.9,118.1,97.0,3.2,81.4,49701.0,108.7,98.9,37.2,4.8
4,31.0,Azerbaijan,5.173,27.5,156.9,99.7,20.8,72.8,4782.0,121.6,88.3,16.8,6.0


In [4]:
print(happiness_joined_df.shape)

(91, 13)


## Modeling -- Random Forest regression using all 10 indep variables except "country_name", "country_code",  "life_expectancy" & "consumer_price_index"

In [5]:
# select features 
target = "life_ladder"
features = happiness_joined_df.columns.drop([target,"country_code","country_name"])#, "life_expectancy", "consumer_price_index", "democracy_index", "infant_mortality_per1000_births","gender_ratio_males_per100_female", "population_density"])#,"unemployment_rate"])
X = happiness_joined_df[features]
y = happiness_joined_df[target]

# either 5 variables or 10, but with no splits

In [6]:
# Normal train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size=0.92)

In [7]:
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor
 
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
 
# fit the regressor with x and y data
regressor.fit(X, y)


# fitmodel.fit(X_train, y_train)
# predict
y_pred = regressor.predict(X)
from sklearn.preprocessing import StandardScaler
# 3) Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X)

# 4) Scaling the data.
X_train_scaled = X_scaler.transform(X)
X_test_scaled = X_scaler.transform(X)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(83, 10)
(8, 10)
(83,)
(8,)


In [8]:
print(np.mean(X_train_scaled[:,0]))
print(np.mean(X_test_scaled[:,0]))
print(np.std(X_train_scaled[:,0]))
print(np.std(X_test_scaled[:,0]))

-6.417333087393762e-16
-6.417333087393762e-16
1.0
1.0


In [9]:
# model evaluation
# ref: https://towardsdatascience.com/multiple-regression-as-a-machine-learning-algorithm-a98a6b9f307b
from sklearn.metrics import r2_score, mean_squared_error
score10 = r2_score(y, y_pred)
print("R2score:", score10)
rmse10 = mean_squared_error(y, y_pred, squared=False)
print('root mean squared error: ', rmse10)
# ref: https://datagy.io/python-sklearn-linear-regression/

R2score: 0.952963470279522
root mean squared error:  0.2282992674438192


In [10]:
# Make predictions using the test data
y_pred = regressor.predict(X)
results = pd.DataFrame({
    "Prediction": y_pred, 
    "Actual": y
}).reset_index(drop=True)
results.head()

,Prediction,Actual
0,5.08861,4.995
1,4.89217,4.745
2,5.40620,5.488
3,7.17204,7.195
4,5.27392,5.173


In [11]:
# select features 
target = "life_ladder"
features = happiness_joined_df.columns.drop([target,"country_code","country_name", "life_expectancy"])#, "consumer_price_index", "democracy_index", "infant_mortality_per1000_births","gender_ratio_males_per100_female", "population_density"])#,"unemployment_rate"])
X = happiness_joined_df[features]
y = happiness_joined_df[target]

# either 5 variables or 10, but with no splits

In [12]:
# Normal train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size=0.92)
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor
 
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
 
# fit the regressor with x and y data
regressor.fit(X, y)


# fitmodel.fit(X_train, y_train)
# predict
y_pred = regressor.predict(X)
from sklearn.preprocessing import StandardScaler
# 3) Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X)

# 4) Scaling the data.
X_train_scaled = X_scaler.transform(X)
X_test_scaled = X_scaler.transform(X)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
## 
print(np.mean(X_train_scaled[:,0]))
print(np.mean(X_test_scaled[:,0]))
print(np.std(X_train_scaled[:,0]))
print(np.std(X_test_scaled[:,0]))

# model evaluation
# ref: https://towardsdatascience.com/multiple-regression-as-a-machine-learning-algorithm-a98a6b9f307b
from sklearn.metrics import r2_score, mean_squared_error
score9 = r2_score(y, y_pred)
print("R2score:", score9)
rmse9 = mean_squared_error(y, y_pred, squared=False)
print('root mean squared error: ', rmse9)
# ref: https://datagy.io/python-sklearn-linear-regression/

(83, 9)
(8, 9)
(83,)
(8,)
-6.417333087393762e-16
-6.417333087393762e-16
1.0
1.0
R2score: 0.9546450601342129
root mean squared error:  0.22418119473436515


In [13]:
# select features 
target = "life_ladder"
features = happiness_joined_df.columns.drop([target,"country_code","country_name", "life_expectancy", "consumer_price_index"])#, "democracy_index", "infant_mortality_per1000_births","gender_ratio_males_per100_female", "population_density"])#,"unemployment_rate"])
X = happiness_joined_df[features]
y = happiness_joined_df[target]

# either 5 variables or 10, but with no splits
# Normal train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size=0.92)
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor
 
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
 
# fit the regressor with x and y data
regressor.fit(X, y)


# fitmodel.fit(X_train, y_train)
# predict
y_pred = regressor.predict(X)
from sklearn.preprocessing import StandardScaler
# 3) Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X)

# 4) Scaling the data.
X_train_scaled = X_scaler.transform(X)
X_test_scaled = X_scaler.transform(X)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
## 
print(np.mean(X_train_scaled[:,0]))
print(np.mean(X_test_scaled[:,0]))
print(np.std(X_train_scaled[:,0]))
print(np.std(X_test_scaled[:,0]))

# model evaluation
# ref: https://towardsdatascience.com/multiple-regression-as-a-machine-learning-algorithm-a98a6b9f307b
from sklearn.metrics import r2_score, mean_squared_error
score8 = r2_score(y, y_pred)
print("R2score:", score8)
rmse8 = mean_squared_error(y, y_pred, squared=False)
print('root mean squared error: ', rmse8)
# ref: https://datagy.io/python-sklearn-linear-regression/

(83, 8)
(8, 8)
(83,)
(8,)
-6.417333087393762e-16
-6.417333087393762e-16
1.0
1.0
R2score: 0.9546180851218766
root mean squared error:  0.2242478511075937


In [14]:
# select features 
target = "life_ladder"
features = happiness_joined_df.columns.drop([target,"country_code","country_name", "life_expectancy", "consumer_price_index", "democracy_index"])#, "infant_mortality_per1000_births","gender_ratio_males_per100_female", "population_density"])#,"unemployment_rate"])
X = happiness_joined_df[features]
y = happiness_joined_df[target]

# either 5 variables or 10, but with no splits
# Normal train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size=0.92)
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor
 
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
 
# fit the regressor with x and y data
regressor.fit(X, y)


# fitmodel.fit(X_train, y_train)
# predict
y_pred = regressor.predict(X)
from sklearn.preprocessing import StandardScaler
# 3) Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X)

# 4) Scaling the data.
X_train_scaled = X_scaler.transform(X)
X_test_scaled = X_scaler.transform(X)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
## 
print(np.mean(X_train_scaled[:,0]))
print(np.mean(X_test_scaled[:,0]))
print(np.std(X_train_scaled[:,0]))
print(np.std(X_test_scaled[:,0]))

# model evaluation
# ref: https://towardsdatascience.com/multiple-regression-as-a-machine-learning-algorithm-a98a6b9f307b
from sklearn.metrics import r2_score, mean_squared_error
score7 = r2_score(y, y_pred)
print("R2score:", score7)
rmse7 = mean_squared_error(y, y_pred, squared=False)
print('root mean squared error: ', rmse7)
# ref: https://datagy.io/python-sklearn-linear-regression/

(83, 7)
(8, 7)
(83,)
(8,)
-7.808161931429672e-16
-7.808161931429672e-16
0.9999999999999999
0.9999999999999999
R2score: 0.9537044409102438
root mean squared error:  0.22649391995090123


In [15]:
# select features 
target = "life_ladder"
features = happiness_joined_df.columns.drop([target,"country_code","country_name", "life_expectancy", "consumer_price_index", "democracy_index", "population_density"])#, "infant_mortality_per1000_births","gender_ratio_males_per100_female", "population_density"])#,"unemployment_rate"])
X = happiness_joined_df[features]
y = happiness_joined_df[target]

# either 5 variables or 10, but with no splits
# Normal train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size=0.92)
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor
 
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
 
# fit the regressor with x and y data
regressor.fit(X, y)


# fitmodel.fit(X_train, y_train)
# predict
y_pred = regressor.predict(X)
from sklearn.preprocessing import StandardScaler
# 3) Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X)

# 4) Scaling the data.
X_train_scaled = X_scaler.transform(X)
X_test_scaled = X_scaler.transform(X)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
## 
print(np.mean(X_train_scaled[:,0]))
print(np.mean(X_test_scaled[:,0]))
print(np.std(X_train_scaled[:,0]))
print(np.std(X_test_scaled[:,0]))

# model evaluation
# ref: https://towardsdatascience.com/multiple-regression-as-a-machine-learning-algorithm-a98a6b9f307b
from sklearn.metrics import r2_score, mean_squared_error
score6 = r2_score(y, y_pred)
print("R2score:", score6)
rmse6 = mean_squared_error(y, y_pred, squared=False)
print('root mean squared error: ', rmse6)
# ref: https://datagy.io/python-sklearn-linear-regression/

(83, 6)
(8, 6)
(83,)
(8,)
-7.808161931429672e-16
-7.808161931429672e-16
0.9999999999999999
0.9999999999999999
R2score: 0.9530206967723747
root mean squared error:  0.22816034624075957


In [16]:
# select features 
target = "life_ladder"
features = happiness_joined_df.columns.drop([target,"country_code","country_name", "life_expectancy", "consumer_price_index", "democracy_index", "population_density", "gender_ratio_males_per100_female"])#, "infant_mortality_per1000_births", "population_density"])#,"unemployment_rate"])
X = happiness_joined_df[features]
y = happiness_joined_df[target]

# either 5 variables or 10, but with no splits
# Normal train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size=0.92)
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor
 
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
 
# fit the regressor with x and y data
regressor.fit(X, y)


# fitmodel.fit(X_train, y_train)
# predict
y_pred = regressor.predict(X)
from sklearn.preprocessing import StandardScaler
# 3) Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X)

# 4) Scaling the data.
X_train_scaled = X_scaler.transform(X)
X_test_scaled = X_scaler.transform(X)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
## 
print(np.mean(X_train_scaled[:,0]))
print(np.mean(X_test_scaled[:,0]))
print(np.std(X_train_scaled[:,0]))
print(np.std(X_test_scaled[:,0]))

# model evaluation
# ref: https://towardsdatascience.com/multiple-regression-as-a-machine-learning-algorithm-a98a6b9f307b
from sklearn.metrics import r2_score, mean_squared_error
score5 = r2_score(y, y_pred)
print("R2score:", score5)
rmse5 = mean_squared_error(y, y_pred, squared=False)
print('root mean squared error: ', rmse5)
# ref: https://datagy.io/python-sklearn-linear-regression/

(83, 5)
(8, 5)
(83,)
(8,)
1.8544384587145473e-16
1.8544384587145473e-16
1.0
1.0
R2score: 0.9544845523222993
root mean squared error:  0.22457752481659238


In [17]:
# select features 
target = "life_ladder"
features = happiness_joined_df.columns.drop([target,"country_code","country_name", "life_expectancy", "consumer_price_index", "democracy_index", "population_density", "gender_ratio_males_per100_female", "infant_mortality_per1000_births"])#, "population_density", #,"unemployment_rate"])
X = happiness_joined_df[features]
y = happiness_joined_df[target]

# either 5 variables or 10, but with no splits
# Normal train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size=0.92)
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor
 
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
 
# fit the regressor with x and y data
regressor.fit(X, y)


# fitmodel.fit(X_train, y_train)
# predict
y_pred = regressor.predict(X)
from sklearn.preprocessing import StandardScaler
# 3) Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X)

# 4) Scaling the data.
X_train_scaled = X_scaler.transform(X)
X_test_scaled = X_scaler.transform(X)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
## 
print(np.mean(X_train_scaled[:,0]))
print(np.mean(X_test_scaled[:,0]))
print(np.std(X_train_scaled[:,0]))
print(np.std(X_test_scaled[:,0]))

# model evaluation
# ref: https://towardsdatascience.com/multiple-regression-as-a-machine-learning-algorithm-a98a6b9f307b
from sklearn.metrics import r2_score, mean_squared_error
score4 = r2_score(y, y_pred)
print("R2score:", score4)
rmse4 = mean_squared_error(y, y_pred, squared=False)
print('root mean squared error: ', rmse4)
# ref: https://datagy.io/python-sklearn-linear-regression/

(83, 4)
(8, 4)
(83,)
(8,)
-4.1480860260720135e-17
-4.1480860260720135e-17
1.0
1.0
R2score: 0.9547636678378568
root mean squared error:  0.2238878747213719


In [24]:
# summary_df that shows the line equation of the model: 
# initialize list of lists
data = [['"country_code" & "country_name"', score10,rmse10 ], ['same as above &  "life_expectancy"', score9,rmse9], ['same as above & "consumer_price_index"', score8,rmse8],['same as above & "population_density"', score7,rmse7],['same as above & "gender_ratio_males_per100_female"', score6,rmse6],['same as above & "democracy_index"', score5,rmse5],['same as above & infant_mortality_per1000_births', score4,rmse4]]
 
 
# Create the pandas DataFrame
summary_90_10_df = pd.DataFrame(data, columns = ['without running following x-variables:', 'R-squared_score', 'RMSE_Score' ])

summary_90_10_df.round(5)

,without running following x-variables:,R-squared_score,RMSE_Score
0,"""country_code"" & ""country_name""",0.95296,0.22830
1,"same as above & ""life_expectancy""",0.95465,0.22418
2,"same as above & ""consumer_price_index""",0.95462,0.22425
3,"same as above & ""population_density""",0.95370,0.22649
4,"same as above & ""gender_ratio_males_per100_fem...",0.95302,0.22816
5,"same as above & ""democracy_index""",0.95448,0.22458
6,same as above & infant_mortality_per1000_births,0.95476,0.22389


In [19]:
# https://towardsdatascience.com/super-simple-machine-learning-by-me-multiple-linear-regression-part-1-447800e8b624
# https://www.datacourses.com/find-p-value-significance-in-scikit-learn-3810/
# https://www.w3schools.com/python/python_ml_multiple_regression.asp

# https://www.quora.com/What-are-some-good-machine-learning-algorithms-for-predicting-a-continuous-variable-with-N-number-of-inputs
# https://www.jigsawacademy.com/popular-regression-algorithms-ml/   (imp)


# https://colab.research.google.com/drive/1TY5M27edj_Aeh-vWE42SUMUwv49w-EW5?usp=sharing&authuser=1#scrollTo=Efpl-zrCO27f

In [20]:
# rerun the significant value >0.05 value for the indep cvariables
# seperate with subheadings for each variables 

In [21]:
# select features 
#X = happiness_joined_df[['democracy_index','consumer_price_index','gender_ratio_males_per100_female','infant_mortality_per1000_births','life_expectancy','per_capita_gdp_dollars','population_density','safe_drinking_water_access_pct','seats_held_by_women_pct','unemployment_rate']]
#y = happiness_joined_df["life_ladder"]
# ref: https://www.pluralsight.com/guides/importing-and-splitting-data-into-dependent-and-independent-features-for-ml
# ref: https://towardsdatascience.com/multiple-regression-as-a-machine-learning-algorithm-a98a6b9f307b


target = "life_ladder"
features = happiness_joined_df.columns.drop([target,"country_code","country_name", "life_expectancy", "consumer_price_index", "democracy_index", "infant_mortality_per1000_births","gender_ratio_males_per100_female", "population_density"])#,"unemployment_rate"])
X = happiness_joined_df[features]
y = happiness_joined_df[target]